In [1]:
project_name_list = ["asciinema","autojump","fabric","face_classification","Sublist3r",'bpytop','furl','rich_cli','sqlparse','sshtunnel','textrank4zh']
project_name_list_2 = ['bpytop','furl','rich_cli','sqlparse','sshtunnel','textrank4zh']
project_list_id=[1,3,4,7,8,11,12,13,14,15,16,22,23,28,29,32,33,35,36,38,45,48,53,56,58]

合并data.csv文件

In [8]:
import pandas as pd

# 你的 CSV 文件列表
csv_files = []
fold = 'data_pycg' #pycg方法
fold = 'data' #infercg方法
for name in project_name_list:
    csv_files.append(f"./{fold}/{name}_data.csv") #pycg方法
# for name in project_list_id:
#     csv_files.append("./data/project{}_data.csv".format(name))

# 读取所有 CSV 并合并（没有标题行）
dfs = [pd.read_csv(f, header=None, encoding='utf-8') for f in csv_files]
merged_df = pd.concat(dfs, ignore_index=True)  # 自动重新索引

# 重新设置第一列为 0 开始的唯一标识符
merged_df.iloc[:, 0] = range(len(merged_df))

# 保存合并后的 CSV
merged_df.to_csv(f"./{fold}/merged_data.csv", index=False, header=False, encoding='utf-8-sig')


生成的data.csv转化为离线推理格式的jsonl文件

In [12]:
import csv
import json
def messages_builder_example(system_prompt, user_prompt):
    #messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content":user_prompt}]
    messages = [{"role": "user", "content":system_prompt + user_prompt}]
    return messages

def csv2jsonl(input_file,output_file):
    with open(input_file, "r", encoding='utf-8') as fin:
        with open(output_file, 'w', encoding='utf-8') as fout:
            csvreader = csv.reader(fin)
            for row in csvreader:
                #body = {"model": "qwen-max","temperature":0 ,"messages": messages_builder_example(row[1],row[2])}
                body = {"model": "deepseek-r1","messages": messages_builder_example(row[1],row[2])}
                # 选择Embedding文本向量模型进行调用时，url的值需填写"/v1/embeddings"
                request = {"custom_id": row[0], "method": "POST", "url": "/v1/chat/completions", "body": body}
                fout.write(json.dumps(request, separators=(',', ':'), ensure_ascii=False) + "\n", )

# for name in project_name_list:
#     input_file = "./data/{}_data.csv".format(name)
#     output_file = "./data/{}_data.jsonl".format(name)
#     csv2jsonl(input_file,output_file)

# input_file = "./data/micro-benchmark_data.csv".format(name)
# output_file = "./data/micro-benchmark_data.jsonl".format(name)
#csv2jsonl(input_file,output_file)

fold = 'data_pycg' #pycg方法
#fold = 'data' #infercg方法
input_file = f"./{fold}/merged_data.csv"
output_file = f"./{fold}/merged_data.jsonl"
csv2jsonl(input_file,output_file)

将离线推理的得到的答案(jsonl)转化为result.csv储存

In [1]:
import json
import csv


def jsonl2res(input_file = "result.jsonl",input_file2 = "data.csv",output_file = "result.csv"):
    columns = ["custom_id",
            "model",
            "request_id",
            "status_code",
            "error_code",
            "error_message",
            "created",
            "content",
            "usage",
            "project_name"]

    def dict_get_string(dict_obj, path):
        obj = dict_obj
        try:
            for element in path:
                obj = obj[element]
            return obj
        except:
            return None

    with open(input_file, "r", encoding='utf-8') as fin:
        with open(output_file, 'w', encoding='utf-8', newline='') as fout:
            with open(input_file2, 'r', encoding='utf-8', newline='') as fin2:
                writer = csv.writer(fout)
                writer.writerow(columns)  # 写入列标题行
                reader2 = list(csv.reader(fin2))
                for i,line in enumerate(fin):
                    try:
                        request_result = json.loads(line)
                        row = [
                            dict_get_string(request_result, ["custom_id"]),
                            dict_get_string(request_result, ["response", "body", "model"]),
                            dict_get_string(request_result, ["response", "request_id"]),
                            dict_get_string(request_result, ["response", "status_code"]),
                            dict_get_string(request_result, ["error", "error_code"]),
                            dict_get_string(request_result, ["error", "error_message"]),
                            dict_get_string(request_result, ["response", "body", "created"]),
                            dict_get_string(request_result, ["response", "body", "choices", 0, "message", "content"]),
                            dict_get_string(request_result, ["response", "body", "usage"]),
                            reader2[i][5]
                        ]
                        # 如果某个字段为空，可以替换成默认值（例如空字符串）
                        row = [value if value is not None else "" for value in row]

                        # 如果 row 中所有值都为空，则跳过该行
                        if any(row):
                            writer.writerow(row)
                    except json.JSONDecodeError:
                        print("Invalid JSON in line, skipping.")

# for name in project_name_list:
#     try:
#         input_file = "./data/{}_result.jsonl".format(name)
#         input_file2 = "./data/{}_data.csv".format(name)
#         output_file = "./data/{}_result.csv".format(name)
#         jsonl2res(input_file,input_file2,output_file)
#     except:
#         print(name)
#         pass

#fold = "data"
fold = "data_pycg"
input_file = f"./{fold}/merged_result.jsonl"
input_file2 = f"./{fold}/merged_data.csv"
output_file = f"./{fold}/merged_result.csv"
jsonl2res(input_file,input_file2,output_file)

# input_file = "./data/micro-benchmark_result.jsonl".format(name)
# input_file2 = "./data/micro-benchmark_data.csv".format(name)
# output_file = "./data/micro-benchmark_result.csv".format(name)
# jsonl2res(input_file,input_file2,output_file)


将储存的csv文件，生成调用图

In [4]:
import csv
import re
import json

all_edges = 0
def load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def extract_percentage(text):
    matches = re.findall(r'(?<![-\d])(?:100|[1-9]?\d(?:\.\d)?)%(?!\d)', text)
    percentages = [float(match.rstrip('%')) for match in matches]  # 去掉 '%' 并转换为浮点数
    return min(percentages) if percentages else None  # 返回最小值，如果为空则返回 None

def solve(name,input_file1="data.csv",input_file2="result.csv") :

    project_data = {}
    project_json_path = {}
    
    project_json_path[name] = "../PyCG_data/{}.json".format(name)
    project_data[name] = load_json(project_json_path[name])
    project_data[name] = {}
    if name in project_name_list_2:
        project_json_path[name] = "../file2class_data/{}.json".format(name)
        project_data[name] = load_json(project_json_path[name])

    caller_candidates = []
    with open(input_file1, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            caller = row[3]
            callee = row[4]
            caller_candidates.append({"caller":caller,"callee":callee})
    global all_edges
    all_edges += len(caller_candidates)
    result = [0] * len(caller_candidates)
    project_name_idx = []
    with open(input_file2, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        # 跳过标题行
        next(reader)
        for i,row in enumerate(reader):
            custom_id = int(row[0])
            content = row[7]
            project_name_idx.append(row[9])
            precent = extract_percentage(content)
            if precent >= 75:
                result[i] += 1
            

    for i, call in enumerate(caller_candidates):

        caller, candidate = call["caller"], call["callee"]
        yes_total = result[i]
        name = project_name_idx[i]
        if yes_total:
            if caller in project_data[name]:
                project_data[name][caller].append(candidate)  # 将 candidate 添加到原列表
                # 去重
                project_data[name][caller] = list(set(project_data[name][caller]))
            else:
                project_data[name][caller] = []
                project_data[name][caller].append(candidate)  # 如果 caller 不存在，直接插入


    filename_with_extension = project_json_path[name].split('/')[-1]
    with open("../Ae_data/{}".format(filename_with_extension), "w", encoding="utf-8") as file:
        json.dump(project_data[name], file, indent=4, ensure_ascii=False)

for name in project_name_list:
    #try:
        input_file1 = "./data/{}_data.csv".format(name)
        input_file2 = "./data/{}_result.csv".format(name)
        solve(name,input_file1,input_file2) 


print(f"Total edges processed: {all_edges}")
    #except:
    #    print(name)
    #    pass 
# input_file1 = "./data/micro-benchmark_data.csv"
# input_file2 = "./data/micro-benchmark_result.csv"
# solve('micro-benchmark',input_file1,input_file2) 


Total edges processed: 7342


merged生成调用图

In [4]:
import csv
import re
import json

def load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def extract_percentage(text):
    matches = re.findall(r'(?<![-\d])(?:100|[1-9]?\d(?:\.\d)?)%(?!\d)', text)
    percentages = [float(match.rstrip('%')) for match in matches]  # 去掉 '%' 并转换为浮点数
    return min(percentages) if percentages else None  # 返回最小值，如果为空则返回 None

def solve(input_file1="data.csv",input_file2="result.csv",op="dynamic") :

    project_data = {}
    project_json_path = {}
    
    if op!="dynamic":
        for name in project_name_list:
            project_json_path[name] = "../PyCG_data/{}.json".format(name)
            project_data[name] = load_json(project_json_path[name])
            project_data[name] = {}
            if name in project_name_list_2:
                project_json_path[name] = "../file2class_data/{}.json".format(name)
                project_data[name] = load_json(project_json_path[name])
    else :
        for id in project_list_id:
            name = "project{}".format(id)
            project_json_path[name] = "../PyCG_data/{}.json".format(name)
            project_data[name] = load_json(project_json_path[name])
            project_data[name] = {}
            if name in project_name_list_2:
                project_json_path[name] = "../file2class_data/{}.json".format(name)
                project_data[name] = load_json(project_json_path[name])

    caller_candidates = []
    project_name_idx = []
    with open(input_file1, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            caller = row[3]
            callee = row[4]
            caller_candidates.append({"caller":caller,"callee":callee})
            project_name_idx.append(row[5])

    result = [0] * len(caller_candidates)
    
    #print(len(caller_candidates))
    all_edges = 0
    jian = 0
    with open(input_file2, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        # 跳过标题行
        next(reader)
        for i,row in enumerate(reader):
            custom_id = int(row[0])
            #print(custom_id)
            content = row[7]
            #project_name_idx.append(row[9])
            #project_name_idx[custom_id] = row[9]
            precent = extract_percentage(content)
            all_edges += 1
            if precent == None or precent >= 75:
                result[custom_id] += 1
            # else:
            #     jian += 1
            """随机剪枝"""
            # import random
            # if random.random()>0.44:
            #     result[custom_id] += 1

    for i, call in enumerate(caller_candidates):

        #try:
            caller, candidate = call["caller"], call["callee"]
            yes_total = result[i]
            name = project_name_idx[i]
            if yes_total:
                if caller in project_data[name]:
                    project_data[name][caller].append(candidate)  # 将 candidate 添加到原列表
                    # 去重
                    project_data[name][caller] = list(set(project_data[name][caller]))
                else:
                    project_data[name][caller] = []
                    project_data[name][caller].append(candidate)  # 如果 caller 不存在，直接插入
        # except :
        #     pass
    print(f"prune rate:{jian}/{all_edges}={jian/all_edges:.2%}")
    if op!="dynamic":
        for name in project_name_list:
            filename_with_extension = project_json_path[name].split('/')[-1]
            with open("../Ae_data/{}".format(filename_with_extension), "w", encoding="utf-8") as file:
                json.dump(project_data[name], file, indent=4, ensure_ascii=False)
    else :
        for id in project_list_id:
            name = "project{}".format(id)
            filename_with_extension = project_json_path[name].split('/')[-1]
            with open("../Ae_data/{}".format(filename_with_extension), "w", encoding="utf-8") as file:
                json.dump(project_data[name], file, indent=4, ensure_ascii=False)


fold = "data"
fold = "data_pycg"
input_file1 = f"./{fold}/merged_data.csv"
input_file2 = f"./{fold}/merged_result.csv"
op="dynamic"
op="marco"
solve(input_file1,input_file2,op) 



prune rate:0/3644=0.00%
